# Quantum Circuit Routing Demo
This notebook demonstrates quantum circuit routing using PyZX with different architectures.

In [ ]:
import pyzx as zx
from fractions import Fraction
from pyzx.routing import (
    create_architecture, route_phase_poly, RoutingMethod, ElimMode
)
import matplotlib.pyplot as plt
import networkx as nx

## Create a Simple Quantum Circuit

This section creates a logical quantum circuit on 5 qubits with multiple CNOT gates and a mix of Clifford and non-Clifford ZPhase gates. The gate layout includes long-range interactions that are not natively supported by common hardware architectures, setting up a scenario where routing will need to insert additional operations.


In [ ]:
circ = zx.Circuit(5)

# Add logical interactions that are non-local in line or IBM QX4
circ.add_gate("CNOT", 0, 4)
circ.add_gate("CNOT", 1, 3)
circ.add_gate("CNOT", 4, 2)
circ.add_gate("CNOT", 2, 0)
circ.add_gate("CNOT", 3, 1)
circ.add_gate("CNOT", 0, 1)

# Add a mix of Clifford and T gates
circ.add_gate("ZPhase", 0, Fraction(1, 4))  # T gate
circ.add_gate("ZPhase", 2, Fraction(1, 2))  # Clifford
circ.add_gate("ZPhase", 3, Fraction(3, 4))  # non-Clifford
circ.add_gate("ZPhase", 4, Fraction(1, 8))  # non-Clifford

# Visualize original
zx.draw_matplotlib(circ, figsize=(10, 2), h_edge_draw='box')

## Define and Visualise Architectures

We visualise the logical (unrouted) quantum circuit using PyZX's drawing tools. This gives us a reference for how the circuit looks before being adapted to specific hardware constraints.

In [ ]:
# Convert PyZX GraphS objects to NetworkX graphs
def pyzx_to_nx(pyzx_graph):
    g_nx = nx.Graph()
    for v in pyzx_graph.vertices():
        g_nx.add_node(v)
    for edge in pyzx_graph.edges():
        g_nx.add_edge(edge[0], edge[1])
    return g_nx

# Visualize Architectures
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
nx.draw(pyzx_to_nx(line_arch.graph), with_labels=True, node_color='lightblue', edge_color='gray')
plt.title('Line Architecture')

plt.subplot(1, 2, 2)
nx.draw(pyzx_to_nx(ibm_arch.graph), with_labels=True, node_color='lightgreen', edge_color='gray')
plt.title('IBM QX4 Architecture')

plt.show()


## Route the Circuit

We define two quantum hardware architectures:
- **Line Architecture**: A simple chain of qubits where only neighbors interact.
- **IBM QX4**: A realistic 5-qubit architecture with limited connectivity.
We visualise both to understand the connectivity limitations that will influence routing.

In [ ]:
routed_circ_gray = route_phase_poly(circ, line_arch, method=RoutingMethod.GRAY, mode=ElimMode.STEINER_MODE)
print('Routed Circuit (Gray):', routed_circ_gray.gates)

routed_circ_combined = route_phase_poly(circ, ibm_arch, method=RoutingMethod.GRAY_MEIJER, mode=ElimMode.STEINER_MODE)
print('Routed Circuit (Combined):', routed_circ_combined.gates)

## Visualise Original and Routed Circuits

We apply PyZX’s routing algorithms to map the logical circuit onto each architecture:
- **Gray (Line)**: Routes the circuit to a linear qubit layout.
- **Gray+Meijer (IBM QX4)**: Routes using additional heuristics for the more restrictive IBM QX4 layout.

Routing inserts extra gates to satisfy hardware connectivity constraints.

In [ ]:
def mat22partition(m: Mat2) -> List[Parity]:
    """
    Convert a list of Parity objects into a Mat2 binary matrix.

    :param m: List of Parity objects, each representing a binary row
    :return: Mat2 binary matrix constructed from the partition
    """
    return [Parity(p) for p in m.data]

In [ ]:
print("\nRouted Circuit (Gray) on Line:")
zx.draw_matplotlib(routed_circ_gray, figsize=(10, 2), h_edge_draw='box')

In [ ]:
print("\nRouted Circuit (Combined) on IBM QX4:")
zx.draw_matplotlib(routed_circ_combined, figsize=(10, 2), h_edge_draw='box')

## Benchmark Circuit Statistics

This section outputs a summary of gate counts, CNOT counts, and T-counts for each version of the circuit. It highlights the trade-offs in complexity introduced by routing.


In [ ]:
print('Original Circuit Stats:', circ.stats())
print('Routed Circuit (Gray) Stats:', routed_circ_gray.stats())
print('Routed Circuit (Combined) Stats:', routed_circ_combined.stats())